In [1]:
#!nvidia-smi

In [2]:
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=5

In [3]:
import os
import numpy as np
from scipy.misc import imread, imresize
import tensorflow as tf
import matplotlib.pyplot as plt
import random

tf.set_random_seed(777)

In [4]:
### Modify !!! #############
save_filename = 'jwlee.ckpt'
############################

In [5]:
img_width = 28
img_height = 28

tfrecord_train = 'fashion_train.tfrecord'
tfrecord_val = 'fashion_val.tfrecord'
tfrecord_dir = 'tfrecords'

In [6]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
n_train = 50000
n_val = 10000
n_class = 10

In [7]:
def read_and_decode(filename_queue, n_batch):
    
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    
    features = tf.parse_single_example(
        serialized_example,
        features={            
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64)
        })
    
    # Convert from a scalar string tensor
    image = tf.decode_raw(features['image'], tf.uint8)        
    label = tf.cast(features['label'], tf.int32)
    label_onehot = tf.one_hot(label, depth=n_class)
    
    image = tf.reshape(image, [img_height, img_width, 1])    
    
    images, labels = tf.train.batch([image, label_onehot],
                                           batch_size=n_batch,
                                           capacity=10000,
                                           num_threads=4)
    
    return images, labels

In [8]:
X = tf.placeholder(tf.float32, [None, img_height, img_width, 1])
Y = tf.placeholder(tf.float32, [None, n_class])
is_train = tf.placeholder(tf.bool)

In [9]:
conv1 = tf.layers.conv2d(inputs=X, filters=32, kernel_size=[3, 3],
                        padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                        padding="SAME", strides=2)
drop1 = tf.layers.dropout(inputs=pool1, rate=0.3, training=is_train)

conv2 = tf.layers.conv2d(inputs=drop1, filters=64, kernel_size=[3, 3],
                        padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                        padding="SAME", strides=2)
drop2 = tf.layers.dropout(inputs=pool2, rate=0.3, training=is_train)

conv3 = tf.layers.conv2d(inputs=drop2, filters=128, kernel_size=[3, 3],
                        padding="SAME", activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],
                        padding="SAME", strides=2)
drop3 = tf.layers.dropout(inputs=pool3, rate=0.3, training=is_train)

In [10]:
flat3 = tf.contrib.layers.flatten(drop3)
dense4 = tf.layers.dense(inputs=flat3, units=625, activation=tf.nn.relu)
drop4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=is_train)

logits = tf.layers.dense(inputs=drop4, units=10)

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [12]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
cwd = os.getcwd()
train_path = os.path.join(cwd, '..', tfrecord_dir, tfrecord_train)
val_path = os.path.join(cwd, '..', tfrecord_dir, tfrecord_val)

In [14]:
filename_queue = tf.train.string_input_producer([train_path], num_epochs=training_epochs)
image_batch, label_batch = read_and_decode(filename_queue, batch_size)
filename_queue_val = tf.train.string_input_producer([val_path], num_epochs=training_epochs)
image_val, label_val = read_and_decode(filename_queue_val, batch_size)

In [15]:
train_var = [X, Y, is_train, logits, accuracy]
tf.add_to_collection('train_var', train_var[0])
tf.add_to_collection('train_var', train_var[1])
tf.add_to_collection('train_var', train_var[2])
tf.add_to_collection('train_var', train_var[3])
tf.add_to_collection('train_var', train_var[4])
saver = tf.train.Saver()
##saver.export_meta_graph(os.path.join(cur_dir, 'checkpoints', 'mnist_ckpt.meta'), collection_list=['train_var'])

In [16]:
init_op = tf.group(tf.global_variables_initializer(),
                      tf.local_variables_initializer())
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
sess.run(init_op)

In [17]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0.
    avg_train_acc = 0.
    avg_val_acc = 0.
    total_batch = int(n_train / batch_size)
    total_batch_val = int(n_val / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = sess.run([image_batch, label_batch])
        batch_xs = batch_xs/255.
        feed_dict = {X: batch_xs, Y: batch_ys, is_train: True}
        #feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        acc, c, _ = sess.run([accuracy, cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        avg_train_acc += acc / total_batch
        
    for i in range(total_batch_val):
        batch_xs, batch_ys = sess.run([image_val, label_val])
        batch_xs = batch_xs/255.
        feed_dict = {X: batch_xs, Y: batch_ys, is_train: False}
        acc = sess.run(accuracy, feed_dict=feed_dict)
        avg_val_acc += acc / total_batch_val

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 'train accuracy = ', 
         '{:.5f}'.format(avg_train_acc), 'validation accuracy = ', '{:.5f}'.format(avg_val_acc))


print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 0001 cost = 0.558233670 train accuracy =  0.79436 validation accuracy =  0.87260
Epoch: 0002 cost = 0.371824895 train accuracy =  0.86396 validation accuracy =  0.88220
Epoch: 0003 cost = 0.323153757 train accuracy =  0.88138 validation accuracy =  0.88870
Epoch: 0004 cost = 0.294441740 train accuracy =  0.89188 validation accuracy =  0.89600
Epoch: 0005 cost = 0.273505639 train accuracy =  0.89906 validation accuracy =  0.90090
Epoch: 0006 cost = 0.254130315 train accuracy =  0.90570 validation accuracy =  0.91080
Epoch: 0007 cost = 0.240963971 train accuracy =  0.91034 validation accuracy =  0.91460
Epoch: 0008 cost = 0.227920539 train accuracy =  0.91488 validation accuracy =  0.90950
Epoch: 0009 cost = 0.217932968 train accuracy =  0.91980 validation accuracy =  0.91710
Epoch: 0010 cost = 0.209030790 train accuracy =  0.92104 validation accuracy =  0.91940
Epoch: 0011 cost = 0.198313599 train accuracy =  0.92560 validation accuracy =  0.9

In [18]:
saver.save(sess, os.path.join(cwd, '..', 'checkpoints', save_filename))

'c:\\Work\\FC_TF_course\\..\\checkpoints\\jwlee3.ckpt'

In [19]:
coord.request_stop()
coord.join(threads) 